In [1]:
import pandas as pd
import numpy as np

import re
import string

import spacy
#!pip install --upgrade gensim
import gensim
from gensim import corpora

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:

# libraries for visualization
!pip install pyldavis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for pyldavis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=a8fadc9147fd91d23bbae9ae576f08e02dcb961b2f34946e5c1ddc2a7c070a3f
  Stored in directory: /root/.cache/pip/wheels/90/61/ec/9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=15333359ddd2a11279c8efdb948734769ce265793960d2f7b31a1fe4a3ca3831
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built pyldavis sklearn


In [3]:
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable


In [4]:
from google.colab import drive
drive.mount('/content/drive')
review_data = pd.read_csv('./drive/MyDrive/Colab Notebooks/tripadvisor_hotel_reviews.csv')
review_data.head()
#print(review_data.head(2))
#print(len(review_data))
#print('Unique Products')
#print(len(review_data.groupby('ProductId')))
#print('Unique Users')
#print(len(review_data.groupby('UserId')))

Mounted at /content/drive


,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


In [5]:
def clean_text(text ): 
    delete_dict = {sp_character: '' for sp_character in string.punctuation} 
    delete_dict[' '] = ' ' 
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    #print('cleaned:'+text1)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>3))]) 
    
    return text2.lower()

In [6]:
import nltk
nltk.download('stopwords') # run this one time

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:

review_data.dropna(axis = 0, how ='any',inplace=True) 

In [8]:
review_data.head()

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


In [11]:
review_data['Review'] = review_data['Review'].apply(clean_text)
review_data['Num_words_text'] = review_data['Review'].apply(lambda x:len(str(x).split())) 

In [12]:
review_data.head()

,Review,Rating,Num_words_text
0,nice hotel expensive parking good deal stay ho...,4,79
1,nothing special charge diamond member hilton d...,2,219
2,nice rooms experience hotel monaco seattle goo...,3,170
3,unique great stay wonderful time hotel monaco ...,5,80
4,great stay great stay went seahawk game awesom...,5,166


In [13]:
max_review_data_sentence_length  = review_data['Num_words_text'].max()

mask = (review_data['Num_words_text'] < 100) & (review_data['Num_words_text'] >=20)
df_short_reviews = review_data[mask]
     

In [14]:
df_short_reviews.head()

,Review,Rating,Num_words_text
0,nice hotel expensive parking good deal stay ho...,4,79
3,unique great stay wonderful time hotel monaco ...,5,80
6,cozy stay rainy city husband spent nights mona...,5,94
7,excellent staff housekeeping quality hotel cho...,4,80
8,hotel stayed hotel monaco cruise rooms generou...,5,54


In [15]:
len(review_data.index)

20491

In [16]:
len(df_short_reviews.index)

14043

In [18]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# function to remove stopwords
def remove_stopwords(text):
    textArr = text.split(' ')
    rem_text = " ".join([i for i in textArr if i not in stop_words])
    return rem_text

# remove stopwords from the text
df_short_reviews['Review']=df_short_reviews['Review'].apply(remove_stopwords)

<ipython-input-18-92e9f836886b>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_short_reviews['Review']=df_short_reviews['Review'].apply(remove_stopwords)


In [19]:
df_short_reviews.head()

,Review,Rating,Num_words_text
0,nice hotel expensive parking good deal stay ho...,4,79
3,unique great stay wonderful time hotel monaco ...,5,80
6,cozy stay rainy city husband spent nights mona...,5,94
7,excellent staff housekeeping quality hotel cho...,4,80
8,hotel stayed hotel monaco cruise rooms generou...,5,54


In [20]:

#nlp = spacy.load('en_core_web_md', disable=['parser', 'ner'])
import en_core_web_sm
nlp = en_core_web_sm.load()
def lemmatization(texts,allowed_postags=['NOUN', 'ADJ']): 
       output = []
       for sent in texts:
             doc = nlp(sent) 
             output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags ])
       return output

In [22]:

text_list=df_short_reviews['Review'].tolist()
print(text_list[1])
tokenized_reviews = lemmatization(text_list)
print(tokenized_reviews[1])

unique great stay wonderful time hotel monaco location excellent short stroll main downtown shopping area friendly room showed signs animal hair smells monaco suite sleeping area striped curtains pulled closed nice touch felt cosy goldfish named brandi enjoyed partake free wine coffeetea service lobby thought great feature great staff friendly free wireless internet hotel worked suite laptops decor lovely eclectic pattens color palatte animal print bathrobes feel like rock stars nice look like sterile chain hotel hotel personality excellent stay
['unique', 'great', 'wonderful', 'time', 'hotel', 'location', 'excellent', 'short', 'stroll', 'main', 'downtown', 'shopping', 'area', 'friendly', 'room', 'sign', 'animal', 'hair', 'area', 'curtain', 'nice', 'touch', 'cosy', 'brandi', 'partake', 'free', 'wine', 'coffeetea', 'service', 'lobby', 'great', 'feature', 'great', 'staff', 'friendly', 'free', 'wireless', 'internet', 'hotel', 'lovely', 'eclectic', 'patten', 'color', 'palatte', 'animal', '

In [23]:
dictionary = corpora.Dictionary(tokenized_reviews)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokenized_reviews]

In [24]:
# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=7, random_state=100,
                chunksize=1000, passes=50,iterations=250)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt